In [187]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common import NoSuchElementException
from selenium.common import TimeoutException
import numpy as np
from selenium.webdriver import ActionChains
import pandas as pd
import re

In [184]:
url='https://www.google.com/search?rlz=1C1GCEA_enHR1085HR1085&q=Casablanca%20Reviews&rflfq=1&num=20&stick=H4sIAAAAAAAAAONgkxI2sTQzMjUzsDC2NDUzMTMzN7W03MDI-IpRyDmxODEpJzEvOVEhKLUsM7W8eBErFkEAULFNy0QAAAA&rldimm=4962560839564667599&tbm=lcl&hl=en&sa=X&ved=0CAYQ5foLahcKEwiAzYbJzIKEAxUAAAAAHQAAAAAQCg&biw=1536&bih=729&dpr=1.25#lkt=LocalPoiReviews&arid=ChZDSUhNMG9nS0VJQ0FnSURSNWJlRUhnEAE'

In [189]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

driver.get(url)

AcceptCookies = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept all')]")))
AcceptCookies.click()
driver.maximize_window()

In [188]:
driver.quit()


In [190]:
ReviewsTab = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'SVWlSe') and contains(@class, 't35a5d')]//span[text()='Reviews']")))
ReviewsTab.click()

In [191]:
total_reviews_element=driver.find_element(By.XPATH, "//*[@id='akp_tsuid_13']/div/div[1]/div/g-sticky-content-container/div/block-component/div/div[1]/div/div/div/div[1]/div/div/div[6]/g-flippy-carousel/div/div/ol/li[3]/span/div/div/div/div[1]/div/div[3]/div/div/div[1]/div[2]/div/div/span/span")
total_reviews_text = total_reviews_element.text
total_reviews = int(total_reviews_text.split()[0])

In [ ]:
%%sql


In [192]:
# Initialize a variable to store the text of the last review from the previous iteration
previous_last_review_text = None

while True:
    review_elements = driver.find_elements(By.XPATH, "//*[@data-hveid='2']/following-sibling::*//div[@class='OA1nbd']")
    if not review_elements:
        break  # Break if no reviews are found

    last_element = review_elements[-1]
    last_element.location_once_scrolled_into_view
    time.sleep(2)  # Waiting for potential loading of new reviews

    current_last_review_text = last_element.text.strip()

    # Check if the last review text is the same as in the previous iteration
    if current_last_review_text == previous_last_review_text:
        break  # Break the loop if we are seeing the same last review as before

    previous_last_review_text = current_last_review_text  # Update the last review text for the next iteration

# Continue with the rest of your code...


In [193]:
first_review = driver.find_element(By.XPATH, "//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[1]//div[@class='OA1nbd']")
first_review.location_once_scrolled_into_view
driver.execute_script("arguments[0].scrollIntoView();", first_review)


In [194]:
# Fetch all review elements
review_elements_xpath = "//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div//div[@class='OA1nbd']"
review_elements = driver.find_elements(By.XPATH, review_elements_xpath)

# First pass: Click all 'Translate' buttons
for i in range(1, len(review_elements) + 1):
    translate_button_xpath = f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i}]/div[3]//button[contains(@class, 'qGGAec') and .//span[contains(text(), 'Translated by Google')]]"
    translate_buttons = driver.find_elements(By.XPATH, translate_button_xpath)
    if translate_buttons:
        translate_buttons[0].click()
        #time.sleep(1)  # Adjust timing as necessary

In [129]:
# Second pass: Click all 'More' buttons
for i in range(1, len(review_elements) + 1):
    more_button_xpath = f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i}]//div[@class='OA1nbd']//a[@role='button' and contains(text(), 'More')]"
    more_buttons = driver.find_elements(By.XPATH, more_button_xpath)
    if more_buttons and more_buttons[0].is_displayed():
        driver.execute_script("arguments[0].scrollIntoView();", more_buttons[0])
        time.sleep(0.5)  # Allow time for scrolling
        try:
            more_buttons[0].click()
            time.sleep(0.5)  # Adjust timing as necessary
        except Exception as e:
            print(f"Skipping a non-interactable 'More' button at index {i}: {e}")


In [158]:
def extract_rating(aria_label):
    match = re.search(r"Rated (\d+(\.\d+)?) out of 5", aria_label)
    return float(match.group(1)) if match else None

In [ ]:
# Initialize lists for reviews and ratings
reviews = []
ratings = []

# Start of the main scraping process
previous_last_review_text = None

# Scrape the first review and rating as a starting point
first_review = driver.find_element(By.XPATH, "//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[1]//div[@class='OA1nbd']")
first_review_rating_element = driver.find_element(By.XPATH, "//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[1]//div[contains(@aria-label, 'Rated') and contains(@role, 'img')]")

reviews.append(first_review.text.strip())
ratings.append(extract_rating(first_review_rating_element.get_attribute('aria-label')))

# Main loop to scrape reviews and ratings
while True:
    review_elements = driver.find_elements(By.XPATH, "//*[@data-hveid='2']/following-sibling::*//div[@class='OA1nbd']")

    if not review_elements:
        break  # Break if no reviews are found

    last_element = review_elements[-1]
    driver.execute_script("arguments[0].scrollIntoView();", last_element)

    # Refresh the list of review elements after scrolling
    review_elements = driver.find_elements(By.XPATH, "//*[@data-hveid='2']/following-sibling::*//div[@class='OA1nbd']")
    rating_elements = driver.find_elements(By.XPATH, "//*[@data-hveid='2']/following-sibling::*//div[contains(@aria-label, 'Rated') and contains(@role, 'img')]")

    current_last_review_text = review_elements[-1].text.strip()

    # Check if the last review text is the same as in the previous iteration
    if current_last_review_text == previous_last_review_text:
        break  # Break the loop if we are seeing the same last review as before

    for review_element, rating_element in zip(review_elements, rating_elements):
        review_text = review_element.text.strip()
        rating_text = rating_element.get_attribute('aria-label')

        if review_text and (review_text not in reviews):
            reviews.append(review_text)
            ratings.append(extract_rating(rating_text))

    previous_last_review_text = current_last_review_text

# Print the reviews and ratings for confirmation
for review, rating in zip(reviews, ratings):
    print(f"Review: {review}, Rating: {rating}")